#EE793 Project
##Homomorphic Encryption  

### Koshti Akshata Vaibhav 200070034
### Sinnarkar Darshan Vishnu 20D070079
### Sharvari Ashok Medhe 20D070073

Cryptography serves as a critical tool in preventing data leaks. One particularly significant advancement in this field is homomorphic encryption, which allows for computations to be performed on encrypted data. This technology has gained increasing relevance, especially in light of the growing rates of cloud adoption. In this project, we will explore the integration of homomorphic encryption into a facial recognition pipeline using TenSEAL.


We will be using Deepface for face recognition.
We will encrypt 3 images and we will calculate the squared euclidian distance between the two tensors of these images pairwise and based on the euclidian distance, we will decide if the two images are of same person or not.

In [ ]:
#!pip install tenseal
#!pip install deepface

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import tenseal as ts
from deepface import DeepFace
import base64
from PIL import Image

Loading, displaying of the images:

In [ ]:
img1_path = "/content/drive/MyDrive/EE793/img1.jpg"
img2_path = "/content/drive/MyDrive/EE793/img2.jpg"
img3_path = "/content/drive/MyDrive/EE793/img3.jpg"


image1 = Image.open(img1_path)
image2 = Image.open(img2_path)
image3 = Image.open(img3_path)

# Assuming all images have the same dimensions
total_width = image1.width + image2.width + image3.width
max_height = max(image1.height, image2.height, image3.height)

composite_image = Image.new('RGB', (total_width, max_height))
composite_image.paste(image1, (0, 0))
composite_image.paste(image2, (image1.width, 0))
composite_image.paste(image3, (image1.width + image2.width, 0))

# Save the composite image to a file
composite_image.save('/content/composite_image.jpg')

# Display the saved image file
composite_image_path = '/content/composite_image.jpg'
Image.open(composite_image_path)


Output hidden; open in https://colab.research.google.com to view.

We will calculate facial embeddings for two images using the FaceNet model provided by the DeepFace library. Here's a brief explanation:


DeepFace.represent(img1_path, model_name='Facenet'): This function call extracts facial embeddings from the image located at img1_path using the FaceNet model. It returns a list containing a dictionary with information about the face in the image, including the embedding vector. By accessing [0]['embedding'], we retrieve the embedding vector associated with the face detected in the image.

In [ ]:
img1_embedding = DeepFace.represent(img1_path, model_name = 'Facenet')[0]['embedding']
img2_embedding = DeepFace.represent(img2_path, model_name = 'Facenet')[0]['embedding']
img3_embedding = DeepFace.represent(img3_path, model_name = 'Facenet')[0]['embedding']

Defining two functions for writing and reading data to/from a file. Here's a brief explanation:

1. `write_data(file_name, data)`: This function takes a file name and data as input. If the data is of type bytes, it encodes it to base64 format using `base64.b64encode()`, then writes the encoded data to the specified file in binary mode ('wb').

2. `read_data(file_name)`: This function reads data from the specified file in binary mode ('rb'). It then decodes the data from base64 format to bytes using `base64.b64decode()` and returns the decoded bytes.

Overall, these functions facilitate the encoding of data to base64 before writing to a file and decoding from base64 after reading from a file, ensuring compatibility and integrity of the data.

In [ ]:
def write_data(file_name, data):
    if type(data) == bytes:
        #bytes to base64
        data = base64.b64encode(data)

    with open(file_name, 'wb') as f:
        f.write(data)

def read_data(file_name):
    with open(file_name, 'rb') as f:
        data = f.read()
    #base64 to bytes
    return base64.b64decode(data)

Below code segment initializes a cryptographic context for homomorphic encryption using the CKKS scheme (Cheon-Kim-Kim-Song), a popular choice for practical implementations due to its ability to handle approximate computations on real-valued data. Here's a breakdown of each line:

1. `context = ts.context(ts.SCHEME_TYPE.CKKS, poly_modulus_degree = 8192, coeff_mod_bit_sizes = [60, 40, 40, 60])`: This line creates a context object for the CKKS scheme with the following parameters:
   - `poly_modulus_degree = 8192`: Specifies the degree of the polynomial modulus, which determines the size of the ciphertext space. A larger degree generally allows for higher precision but increases computational complexity.
   - `coeff_mod_bit_sizes = [60, 40, 40, 60]`: Defines the bit sizes for the coefficient modulus. The coefficient modulus is a set of prime numbers that control the noise growth during homomorphic operations. The sizes specified here indicate the bit lengths of these primes, with decreasing values indicating a trade-off between precision and security.

2. `context.generate_galois_keys()`: This line generates Galois keys, which are necessary for performing certain homomorphic operations like rotations on encrypted data. These keys allow the homomorphic encryption system to operate efficiently on encrypted data in a way that preserves the underlying mathematical operations.

3. `context.global_scale = 2**40`: Sets the global scale parameter, which determines the scaling factor used in encoding and decoding plaintext values. Scaling is crucial for managing the precision of computations in homomorphic encryption. By setting the global scale, you establish a reference point for the numerical range of plaintext values, ensuring that operations remain within a manageable range. In this case, the global scale is set to \(2^{40}\), which means that plaintext values will be scaled by \(2^{40}\) during encoding and decoding operations.

In [ ]:
context = ts.context(ts.SCHEME_TYPE.CKKS, poly_modulus_degree = 8192, coeff_mod_bit_sizes = [60, 40, 40, 60])
context.generate_galois_keys()
context.global_scale = 2**40

This code segment involves serialization and storage of cryptographic context objects for the purpose of separating secret and public components. Here's a concise explanation:

1. `secret_context = context.serialize(save_secret_key=True)`: This line serializes the cryptographic context object `context`, saving the secret key along with other parameters. The serialized context, which includes the secret key, is then stored in a file named 'secret.txt' using the `write_data` function.

2. `context.make_context_public()`: This line modifies the original context object by removing the secret key, effectively making it a public context. This operation is often done to ensure that sensitive information (the secret key) is not accidentally leaked or accessed by unauthorized parties.

3. `public_context = context.serialize()`: After making the context public, this line serializes the modified context (now lacking the secret key) and stores it in a file named 'public.txt' using the `write_data` function.

Overall, this code separates the secret and public components of the cryptographic context, storing them in separate files for appropriate usage and security management.

In [ ]:
secret_context = context.serialize(save_secret_key = True)
write_data(file_name='/content/drive/MyDrive/EE793/secret.txt', data = secret_context)

context.make_context_public() #drop the secret_key from the context
public_context = context.serialize()
write_data(file_name='/content/drive/MyDrive/EE793/public.txt', data = public_context)

In [ ]:
del context, secret_context, public_context

##Encryption


This code segment loads previously serialized cryptographic context from a file containing the secret key. Then, it encrypts two vectors using the loaded context. After encryption, it serializes the encrypted vectors and writes them to separate files. Here's a concise explanation:

1. `context = ts.context_from(secret.txt')`: This line reads the serialized context containing the secret key from the file 'secret.txt' using the `read_data` function and reconstructs the cryptographic context object `context` from it.

2. `enc_v1 = ts.ckks_vector(context, img1_embedding)`: This line encrypts the vector `img1_embedding` using the CKKS scheme and the cryptographic context `context`, creating an encrypted vector `enc_v1`.

3. `enc_v1_proto = enc_v1.serialize()`: The encrypted vector `enc_v1` is serialized into a binary format, `enc_v1_proto`, suitable for storage or transmission.

4. `write_data('enc_v1.txt', enc_v1_proto)`: The serialized encrypted vector `enc_v1_proto` is written to a file named 'enc_v1.txt' using the `write_data` function.

Overall, this code encrypts two vectors using a loaded cryptographic context with the secret key, serializes the encrypted vectors, and stores them in separate files for later use or transmission.

In [ ]:
context = ts.context_from(read_data('/content/drive/MyDrive/EE793/secret.txt'))

enc_v1 = ts.ckks_vector(context, img1_embedding)
enc_v2 = ts.ckks_vector(context, img2_embedding)
enc_v3 = ts.ckks_vector(context, img3_embedding)

enc_v1_proto = enc_v1.serialize()
enc_v2_proto = enc_v2.serialize()
enc_v3_proto = enc_v3.serialize()

write_data('/content/drive/MyDrive/EE793/enc_v1.txt', enc_v1_proto)
write_data('/content/drive/MyDrive/EE793/enc_v2.txt', enc_v2_proto)
write_data('/content/drive/MyDrive/EE793/enc_v3.txt', enc_v3_proto)

In [ ]:
del context, enc_v1, enc_v2, enc_v3, enc_v1_proto, enc_v2_proto, enc_v3_proto

# Run the server code before running Decryption code

##Decryption

This code decrypts the euclidian distance files (generated by server) using secret key

In [ ]:
#client has the secret key
context = ts.context_from(read_data('/content/drive/MyDrive/EE793/secret.txt'))

#load euclidean squared value
euclidean_squared_proto_12 = read_data('/content/drive/MyDrive/EE793/euclidean_squared_12.txt')
euclidean_squared_12 = ts.lazy_ckks_vector_from(euclidean_squared_proto_12)
euclidean_squared_12.link_context(context)

#load euclidean squared value
euclidean_squared_proto_23 = read_data('/content/drive/MyDrive/EE793/euclidean_squared_23.txt')
euclidean_squared_23 = ts.lazy_ckks_vector_from(euclidean_squared_proto_23)
euclidean_squared_23.link_context(context)

#load euclidean squared value
euclidean_squared_proto_13 = read_data('/content/drive/MyDrive/EE793/euclidean_squared_13.txt')
euclidean_squared_13 = ts.lazy_ckks_vector_from(euclidean_squared_proto_13)
euclidean_squared_13.link_context(context)

#decrypt it
euclidean_squared_plain = []
euclidean_squared_plain.append(euclidean_squared_12.decrypt()[0])
euclidean_squared_plain.append(euclidean_squared_23.decrypt()[0])
euclidean_squared_plain.append(euclidean_squared_13.decrypt()[0])

Since it is a pre-trained model, the threshold for squared euclidean distance is 100

In [ ]:
if euclidean_squared_plain[0] < 100:
    print('Person 1 & 2 are same')
else:
    print('Person 1 & 2 are different')

if euclidean_squared_plain[1] < 100:
    print('Person 2 & 3 are same')
else:
    print('Person 2 & 3 are different')

if euclidean_squared_plain[2] < 100:
    print('Person 1 & 3 are same')
else:
    print('Person 1 & 3 are different')

Person 1 & 2 are same
Person 2 & 3 are different
Person 1 & 3 are different


In [ ]:
print(euclidean_squared_plain)

[-1.676397113602978e+19, 9.03921698411364e+18, 2.0987992859422786e+19]
